In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
folder_path = "/content/drive/MyDrive/Colab Notebooks/TestSAWO/"
control_path = folder_path + "control/"
dataset_path = folder_path + "datasets/"

run_path = folder_path + "runs/"

# CAN CHANGE THE FOLLOWING
dataset_id = 1

num_runners = 4 # keep this at 4 to ensure maximum parallelization
num_per_runner = 15
pop_size = num_runners * num_per_runner # 4 x 15 would be 60
num_init_per_runner = num_per_runner * 2 # initial population of 120

num_gens = 10

num_create = int(0.3 * pop_size)
num_cross = int(0.0 * pop_size)
cross_k = 2
num_mutate = int(0.5 * pop_size)
mutate_k = 2
num_reprod = pop_size - num_create - num_cross - num_mutate
# CHANGE ANYTHING ELSE AT YOUR OWN RISK


In [ ]:
import sys
sys.path.insert(0,"/content/drive/MyDrive/Colab Notebooks/TestSAWO/")

from Rule import Rule
from RuleSet import RuleSet
from Node import Node
from Architecture import Architecture
from SawoNN import SawoNN

In [ ]:
import json
import time
import copy
import math
import random
import statistics as stats

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

seed = int(random.random() * 123456)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

print("Random seed:", seed)

rs = RuleSet(folder_path + "grammar.txt")
print(rs)

Random seed: 56411
RULE SET
--------
START := { "a": SIZE , "b": SIZE , "c": SIZE } $ self.gu ( 'io' , CONSTANT , 'mse' , ARCHITECTURE_IO ) ~ self.gu ( 'ia' , CONSTANT , 'mse' , ARCHITECTURE_IA ) ~ self.gu ( 'ib' , CONSTANT , 'mse' , ARCHITECTURE_IB ) ~ self.gu ( 'ic' , CONSTANT , 'mse' , ARCHITECTURE_IC ) ~ self.gu ( 'ab' , CONSTANT , 'mse' , ARCHITECTURE_AB ) ~ self.gu ( 'ba' , CONSTANT , 'mse' , ARCHITECTURE_BA ) ~ self.gu ( 'ac' , CONSTANT , 'mse' , ARCHITECTURE_AC ) ~ self.gu ( 'ca' , CONSTANT , 'mse' , ARCHITECTURE_CA ) ~ self.gu ( 'bc' , CONSTANT , 'mse' , ARCHITECTURE_BC ) ~ self.gu ( 'cb' , CONSTANT , 'mse' , ARCHITECTURE_CB ) ~ self.gu ( 'ao' , CONSTANT , 'mse' , ARCHITECTURE_AO ) ~ self.gu ( 'bo' , CONSTANT , 'mse' , ARCHITECTURE_BO ) ~ self.gu ( 'co' , CONSTANT , 'mse' , ARCHITECTURE_CO ) $ ARCHITECTURE
SIZE := 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 | 13 | 14 | 15 | 16 | 17 | 18 | 19 | 20
ARCHITECTURE := ['i', 'o'] | ['i', 'a', 'o'] | ['i', 'ia', 'o'] | ['i', 'b', 'o'

In [ ]:
from ast import literal_eval

size_pos = '{0:07b}'.format(13)
constant_pos = '{0:07b}'.format(1)
#error_pos = '{0:07b}'.format(0)
arch_encode = 'archit'
rand_encode = '_______'

template = ['0000000', size_pos, size_pos, size_pos]
for i in range(13):
    template.append(constant_pos)
    #template.append(error_pos)
    template.append(arch_encode)
template.append(arch_encode)
print(template)

def weightInArch(weight, arch):
    for i in range(len(arch)-1):
        for j in range(len(arch[i])):
            for k in range(len(arch[i+1])):
              if arch[i][j] == weight[0] and arch[i+1][k] == weight[1]:
                  return True
    return False

template_chromosomes = []

arch_rule = rs.findRule("ARCHITECTURE")
all_archs = [literal_eval(" ".join(o)) for o in arch_rule.outputs]
for a in range(len(all_archs)):
    arch = all_archs[a]
    print(arch)

    #now make a template for this arch
    this_template = template.copy()
    this_template[len(this_template)-1] = '{0:07b}'.format(a)
    print(this_template)

    weights = ["io", "ia", "ib", "ic", "ab", "ba", "ac", "ca", "bc", "cb", "ao", "bo", "co"]

    start = 4
    for w in range(len(weights)):
        this_weight = weights[w]
        #this_start = start + w * 3 + 2
        this_start = start + w * 2 + 1
        if weightInArch(this_weight, arch):
            this_arch_rule = rs.findRule("ARCHITECTURE_" + this_weight.upper())
            this_arch_outputs = [literal_eval(" ".join(o)) for o in this_arch_rule.outputs]
            pos = this_arch_outputs.index(arch)
            this_template[this_start] = '{0:07b}'.format(pos)
        else:
            this_template[this_start] = rand_encode
    
    print(this_template)
    template_chromosomes.append(this_template)
    print()

print("NUMBER OF TEMPLATES:", len(template_chromosomes))
    



['0000000', '0001101', '0001101', '0001101', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', 'archit']
['i', 'o']
['0000000', '0001101', '0001101', '0001101', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000001', 'archit', '0000000']
['0000000', '0001101', '0001101', '0001101', '0000001', '0000000', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '0000001', '_______', '

In [ ]:

def numDiffBits(bitstring1, bitstring2):
    count = 0
    for i in range(len(bitstring1)):
        if(bitstring1[i] != bitstring2[i]):
            count += 1
    return count / len(bitstring1)

def chromToBts(chromosome):
    array = []
    for equ in chromosome:
        for ele in equ:
            array.append(ele)
    array = "".join(array)
    return array

def tourn(fit_arr, k):
    pool = []
    for i in range(k):
        rand = int(random.random()*len(fit_arr))
        while(math.isnan(fit_arr[rand])):
            rand = int(random.random()*len(fit_arr))
        pool.append(rand)
    min_pos = pool[0]
    for i in range(1, k):
        pos = pool[i]
        if(fit_arr[pos] < fit_arr[min_pos]):
            min_pos = pos
    return min_pos

def mutate(chromosome):
    which_codon = int(random.random()*len(chromosome))
    codon_pos = int(random.random()*7)
    
    front = chromosome[which_codon][:codon_pos]
    end = chromosome[which_codon][codon_pos+1:]
    char = '0'
    if(chromosome[which_codon][codon_pos] == '0'):
        char = '1'
        
    chromosome[which_codon] = front + char + end
    return chromosome

def crossover(chrom_1, chrom_2):
    which_codon_1 = int(random.random()*len(chrom_1))
    which_codon_2 = int(random.random()*len(chrom_2))
    codon_pos = int(random.random()*7)
    
    front_1 = chrom_1[which_codon_1][:codon_pos]
    end_1 = chrom_1[which_codon_1][codon_pos:]
    front_2 = chrom_2[which_codon_2][:codon_pos]
    end_2 = chrom_2[which_codon_2][codon_pos:]
    
    chrom_1[which_codon_1] = front_1 + end_2
    chrom_2[which_codon_2] = front_2 + end_1

    return chrom_1, chrom_2

def newChromosomeOld():
    num_codons = 31
    build = []
    for i in range(num_codons):
        build.append("")
        for j in range(7):
            rand_char = int(round(random.random()))
            build[i] += str(rand_char)
    return build

def newChromosome():
    rand_template = int(random.random()*len(template_chromosomes))
    this_template = template_chromosomes[rand_template].copy()
    for i in range(len(this_template)):
        for j in range(len(this_template[i])):
            if this_template[i][j] == "_" or random.random() < 0.05:
                rand_char = int(round(random.random()))
                this_template[i] = this_template[i][:j] + str(rand_char) + this_template[i][j+1:]

    return this_template

def printEssential(rs, chromosome):
    eq = rs.constructEquation(chromosome)
    gnn = SawoNN(eq, None, None, None, None)
    return gnn.printEssentialInfo()


In [ ]:

print("create stages control files")
filename = control_path + "stages.json"
file = open(filename, "w")
stages = {}
stages['seed'] = seed
stages['starting'] = True
stages['initpop'] = False
stages['evaluate'] = False
stages['select_and_alter'] = False
stages['finished'] = False
json.dump(stages, file)
file.close()

def changeState(current):
    stages_file = open(control_path + 'stages.json')
    stages = json.load(stages_file)
    stages_file.close()

    stages['starting'] = False
    stages['initpop'] = False
    stages['evaluate'] = False
    stages['select_and_alter'] = False
    stages['finished'] = False

    stages[current] = True

    stages_file = open(control_path + 'stages.json', 'w')
    json.dump(stages, stages_file)
    stages_file.close()



#CHANGED TO DO INIT POP HERE
print("create per-runner control files")
for i in range(num_runners):
    #init pop status
    filename = control_path + "runner_" + str(i) + "_initpop.json"
    file = open(filename, "w")
    runner = {}
    runner['which_to_init'] = num_init_per_runner
    runner['finished'] = True #THIS WAS: False
    json.dump(runner, file)
    file.close()

    #eval pop status
    filename = control_path + "runner_" + str(i) + "_evalpop.json"
    file = open(filename, "w")
    runner = {}
    runner['which_to_eval'] = num_init_per_runner
    runner['chromosomes'] = [newChromosome() for x in range(i*num_init_per_runner, (i+1)*num_init_per_runner)] #THIS WAS: None instead of newChromosome()
    print("RUNNER", i)
    for c in runner['chromosomes']:
        print(c)
    runner['fitnesses'] = [None for x in range(i*num_init_per_runner, (i+1)*num_init_per_runner)]
    runner['parent_fitnesses'] = [None for x in range(i*num_init_per_runner, (i+1)*num_init_per_runner)]
    runner['gen_ops'] = [None for x in range(i*num_init_per_runner, (i+1)*num_init_per_runner)]
    runner['finished'] = False
    runner['gen'] = 1
    json.dump(runner, file)
    file.close()


print("tell runners to fill initial population")
changeState('initpop')

print("wait for init pops to be finished")
finished = False

count = 0
while not finished:
    time.sleep(random.random() * 3) #seconds

    count += 1
    print("waiting... ", count)

    num_finished = 0
    for i in range(num_runners):
        filename = control_path + "runner_" + str(i) + "_initpop.json"
        file = open(filename)
        runner = json.load(file)
        file.close()

        if(runner['finished']):
            num_finished += 1
    
    print("num_finished", num_finished)
    
    if(num_finished == num_runners):
        finished = True

print("finished!")

best_chromosome = None
best_fit = float('inf')

gens = []
total_mean_arr = []
total_min_arr = []
total_1_4_quart_arr = []
total_2_4_quart_arr = []
total_3_4_quart_arr = []
total_std_arr = []
total_diff_arr = []

reprod_to_nan_arr = []
create_to_nan_arr = []
cross_to_nan_arr = []
mutate_to_nan_arr = []

reprod_improve_arr = []
create_improve_arr = []
cross_improve_arr = []
mutate_improve_arr = []

reprod_decrease_arr = []
create_decrease_arr = []
cross_decrease_arr = []
mutate_decrease_arr = []

member_hash_store = {}

start_time = time.time()

for gen in range(1, num_gens+1):

    gens.append(gen)
    print()
    print()
    print("GENERATION", gen)
    if gen != 1:
        print("BEST SO FAR:", best_chromosome)
        print(printEssential(rs, best_chromosome))
        print(best_fit)
    print()

    print(gen, "- tell runners to evaluate population")
    changeState('evaluate')



    print(gen, "- wait for eval pops to be finished")
    finished = False

    count = 0
    while not finished:
        time.sleep(random.random() * 3) #seconds

        count += 1
        print(gen, "- waiting... ", count)

        num_finished = 0
        for i in range(num_runners):
            filename = control_path + "runner_" + str(i) + "_evalpop.json"
            file = open(filename)
            runner = json.load(file)
            file.close()

            if(runner['finished']):
                num_finished += 1
        
        print("num_finished", num_finished)

        if(num_finished == num_runners):
            finished = True

    print(gen, "- finished!")



    print(gen, "- tell runners to wait for selection")
    changeState('select_and_alter')
  

    print(gen, "! re-seed for this gen:", seed+gen)
    #seed with gen after waiting undecidable num of times
    random.seed(seed + gen)
    np.random.seed(seed + gen)
    tf.random.set_seed(seed + gen)


    print(gen, "- gather fitnesses")
    chrom_arr = []
    fit_arr = []
    parent_fit_arr = []
    gen_op_arr = []

    for i in range(num_runners):
        runner_filename = control_path + "runner_" + str(i) + "_evalpop.json"
        runner_file = open(runner_filename)
        runner = json.load(runner_file)
        runner_file.close()

        chrom_arr.extend(runner['chromosomes'])
        fit_arr.extend(runner['fitnesses'])
        parent_fit_arr.extend(runner['parent_fitnesses'])
        gen_op_arr.extend(runner['gen_ops'])

    print(chrom_arr)
    print(fit_arr)
    print(parent_fit_arr)
    print(gen_op_arr)

    new_fit_arr = []
    new_chrom_arr = []
    new_parent_fit_arr = []
    new_gen_op_arr = []

    reprod_to_nan = 0
    create_to_nan = 0
    cross_to_nan = 0
    mutate_to_nan = 0

    reprod_improve = 0
    create_improve = 0
    cross_improve = 0
    mutate_improve = 0

    reprod_decrease = 0
    create_decrease = 0
    cross_decrease = 0
    mutate_decrease = 0
    
    for i in range(len(fit_arr)):

        if not math.isnan(fit_arr[i]) and not math.isinf(fit_arr[i]):
            new_fit_arr.append(fit_arr[i])
            new_chrom_arr.append(chrom_arr[i])
            new_parent_fit_arr.append(parent_fit_arr[i])
            new_gen_op_arr.append(gen_op_arr[i])

            if gen >= 2:
                if fit_arr[i] < parent_fit_arr[i]:
                    if gen_op_arr[i] == "reproduction":
                        reprod_improve += 1
                    elif gen_op_arr[i] == "creation":
                        create_improve += 1
                    elif gen_op_arr[i] == "crossover":
                        cross_improve += 1
                    elif gen_op_arr[i] == "mutation":
                        mutate_improve += 1
                else:
                    if gen_op_arr[i] == "reproduction":
                        reprod_decrease += 1
                    elif gen_op_arr[i] == "creation":
                        create_decrease += 1
                    elif gen_op_arr[i] == "crossover":
                        cross_decrease += 1
                    elif gen_op_arr[i] == "mutation":
                        mutate_decrease += 1

        else:
            if gen >= 2:
                if gen_op_arr[i] == "reproduction":
                    reprod_to_nan += 1
                elif gen_op_arr[i] == "creation":
                    create_to_nan += 1
                elif gen_op_arr[i] == "crossover":
                    cross_to_nan += 1
                elif gen_op_arr[i] == "mutation":
                    mutate_to_nan += 1
    
    if gen == 1:
        reprod_to_nan_arr.append(0)
        create_to_nan_arr.append(0)
        cross_to_nan_arr.append(0)
        mutate_to_nan_arr.append(0)

        reprod_improve_arr.append(0)
        create_improve_arr.append(0)
        cross_improve_arr.append(0)
        mutate_improve_arr.append(0)

        reprod_decrease_arr.append(0)
        create_decrease_arr.append(0)
        cross_decrease_arr.append(0)
        mutate_decrease_arr.append(0)
    else:
        print()
        print("reprod_to_nan", reprod_to_nan, (reprod_to_nan / (reprod_to_nan + reprod_improve + reprod_decrease)))
        print("reprod_improve", reprod_improve, (reprod_improve / (reprod_to_nan + reprod_improve + reprod_decrease)))
        print("reprod_decrease", reprod_decrease, (reprod_decrease / (reprod_to_nan + reprod_improve + reprod_decrease)))
        
        print("create_to_nan", create_to_nan, (create_to_nan / (create_to_nan + create_improve + create_decrease)))
        print("create_improve", create_improve, (create_improve / (create_to_nan + create_improve + create_decrease)))
        print("create_decrease", create_decrease, (create_decrease / (create_to_nan + create_improve + create_decrease)))

        print("mutate_to_nan", mutate_to_nan, (mutate_to_nan / (mutate_to_nan + mutate_improve + mutate_decrease)))
        print("mutate_improve", mutate_improve, (mutate_improve / (mutate_to_nan + mutate_improve + mutate_decrease)))
        print("mutate_decrease", mutate_decrease, (mutate_decrease / (mutate_to_nan + mutate_improve + mutate_decrease)))
        print()


        reprod_to_nan_arr.append(reprod_to_nan / (reprod_to_nan + reprod_improve + reprod_decrease))
        create_to_nan_arr.append(create_to_nan / (create_to_nan + create_improve + create_decrease))
        mutate_to_nan_arr.append(mutate_to_nan / (mutate_to_nan + mutate_improve + mutate_decrease))

        reprod_improve_arr.append(reprod_improve / (reprod_to_nan + reprod_improve + reprod_decrease))
        create_improve_arr.append(create_improve / (create_to_nan + create_improve + create_decrease))
        mutate_improve_arr.append(mutate_improve / (mutate_to_nan + mutate_improve + mutate_decrease))

        reprod_decrease_arr.append(reprod_decrease / (reprod_to_nan + reprod_improve + reprod_decrease))
        create_decrease_arr.append(create_decrease / (create_to_nan + create_improve + create_decrease))
        mutate_decrease_arr.append(mutate_decrease / (mutate_to_nan + mutate_improve + mutate_decrease))

    
    fit_arr, chrom_arr, parent_fit_arr, gen_op_arr = new_fit_arr, new_chrom_arr, new_parent_fit_arr, new_gen_op_arr
    fit_arr, chrom_arr, parent_fit_arr, gen_op_arr = zip(*sorted(zip(fit_arr, chrom_arr, parent_fit_arr, gen_op_arr)))

    print(chrom_arr)
    print(fit_arr)
    print(parent_fit_arr)
    print(gen_op_arr)

    chrom_arr = chrom_arr[:pop_size]
    fit_arr = fit_arr[:pop_size]
    parent_fit_arr = parent_fit_arr[:pop_size]
    gen_op_arr = gen_op_arr[:pop_size]

    print(chrom_arr)
    print(fit_arr)
    print(parent_fit_arr)
    print(gen_op_arr)

    equation_arr = [printEssential(rs, chromosome) for chromosome in chrom_arr]
    print(equation_arr)

    for i in range(0, int(len(equation_arr)/2)):
        eq = str(equation_arr[i])
        if eq not in member_hash_store:
            new_obj = {"avg": fit_arr[i], "num": 1, "chrom": chrom_arr[i]}
            member_hash_store[eq] = new_obj
        else:
            old_obj = member_hash_store[eq]
            old_avg = old_obj["avg"]
            old_num = old_obj["num"]

            new_num = old_num + 1
            new_avg = ((old_avg * old_num) + fit_arr[i]) / new_num
            new_obj = {"avg": new_avg, "num": new_num, "chrom": chrom_arr[i]}

            member_hash_store[eq] = new_obj

    print()
    history_arr = []
    for key in member_hash_store:
        history_arr.append([key, member_hash_store[key]["avg"], member_hash_store[key]["num"], member_hash_store[key]["chrom"]])
    history_arr = sorted(history_arr, key=lambda x: x[1])
    for i in range(len(history_arr)):
        ele = history_arr[i]
        print(i, ":", ele[0])
        print(ele[1], ele[2], ele[3])
    print()


    x = fit_arr[:int(len(fit_arr)*(3/4))]
    num_bins = 20
    fig, ax = plt.subplots()
    n, bins, patches = ax.hist(x, num_bins)#, density=True)
    ax.set_xlabel('Fitness')
    ax.set_ylabel('Number of individuals')
    ax.set_title('Histogram of fitness')
    fig.tight_layout()
    plt.show()
    plt.cla()
    plt.clf()
    plt.close()

    mean = 0
    std = 0
    if gen > 1:
        mean = stats.mean(x)
        std = stats.stdev(x)
    total_mean_arr.append(mean)
    total_min_arr.append(fit_arr[0])
    total_1_4_quart_arr.append(fit_arr[int((len(fit_arr)/4)*1)])
    total_2_4_quart_arr.append(fit_arr[int((len(fit_arr)/4)*2)])
    total_3_4_quart_arr.append(fit_arr[int((len(fit_arr)/4)*3)])
    total_std_arr.append(std)
    plt.plot(gens, total_min_arr, label="min")
    plt.plot(gens, total_1_4_quart_arr, label="1/4")
    plt.plot(gens, total_2_4_quart_arr, label="2/4")
    plt.plot(gens, total_3_4_quart_arr, label="3/4")
    plt.plot(gens, total_mean_arr, label="mean")
    plt.plot(gens, [a_i + b_i for a_i, b_i in zip(total_mean_arr, total_std_arr)], label="mean+std")
    plt.plot(gens, [a_i - b_i for a_i, b_i in zip(total_mean_arr, total_std_arr)], label="mean-std")
    plt.title('Gens and fitness')
    plt.xlabel('Gens')
    plt.ylabel('Fitness')
    plt.legend()
    plt.show()
    plt.cla()
    plt.clf()
    plt.close()


    plt.plot(gens, reprod_to_nan_arr, label="reprod_to_nan")
    plt.plot(gens, create_to_nan_arr, label="create_to_nan")
    plt.plot(gens, mutate_to_nan_arr, label="mutate_to_nan")
    plt.title('Gens and genop to nan')
    plt.xlabel('Gens')
    plt.ylabel('Genop to nan proportions')
    plt.legend()
    plt.show()
    plt.cla()
    plt.clf()
    plt.close()

    plt.plot(gens, reprod_improve_arr, label="reprod_improve")
    plt.plot(gens, create_improve_arr, label="create_improve")
    plt.plot(gens, mutate_improve_arr, label="mutate_improve")
    plt.title('Gens and genop improve')
    plt.xlabel('Gens')
    plt.ylabel('Genop improve proportions')
    plt.legend()
    plt.show()
    plt.cla()
    plt.clf()
    plt.close()

    plt.plot(gens, reprod_decrease_arr, label="reprod_decrease")
    plt.plot(gens, create_decrease_arr, label="create_decrease")
    plt.plot(gens, mutate_decrease_arr, label="mutate_decrease")
    plt.title('Gens and genop decrease')
    plt.xlabel('Gens')
    plt.ylabel('Genop decrease proportions')
    plt.legend()
    plt.show()
    plt.cla()
    plt.clf()
    plt.close()

    
    total_diff = 0
    count = 0
    for i in range(len(chrom_arr)-1):
        for j in range(i+1, len(chrom_arr)):
            bt1 = chromToBts(chrom_arr[i])
            bt2 = chromToBts(chrom_arr[j])
            diff = numDiffBits(bt1, bt2)
            total_diff += diff
            count += 1
    total_diff /= count
    print("Average bit difference percentage:", total_diff)
    total_diff_arr.append(total_diff)
    plt.plot(gens, total_diff_arr)
    plt.title('Gens and average bitstring diff percentage')
    plt.xlabel('Gens')
    plt.ylabel('Average bit difference percentage')
    plt.show()
    plt.cla()
    plt.clf()
    plt.close()



    for i in range(len(fit_arr)):
        if(not math.isnan(fit_arr[i]) and fit_arr[i] < best_fit):
            best_fit = fit_arr[i]
            best_chromosome = chrom_arr[i]

    print(gen, "- select and alter")
    new_parent_fit_arr = []
    new_chrom_arr = []
    new_equation_arr = []
    new_gen_op_arr = []
    i = 0
    elite_front_pos = 0
    while i < pop_size:
        done = False
        while not done:
            if i < (num_reprod):
                # elitism
                while math.isnan(fit_arr[elite_front_pos]) or math.isinf(fit_arr[elite_front_pos]):
                    elite_front_pos += 1
                choose = elite_front_pos
                chromosome = copy.deepcopy(chrom_arr[choose])
                print("elite reprod:", chrom_arr[choose], fit_arr[choose])
                elite_front_pos += 1

                equation = printEssential(rs, chromosome)
                print("equation:", equation)
                if equation not in new_equation_arr:
                    # reproduction
                    print("insert")
                    new_parent_fit_arr.append(fit_arr[choose])
                    new_gen_op_arr.append("reproduction")
                    new_chrom_arr.append(chromosome)
                    new_equation_arr.append(equation)
                    i += 1
                    done = True

            elif i < (num_reprod + num_create):
                # creation
                chromosome = newChromosome()
                print("create:", chromosome)

                equation = printEssential(rs, chromosome)
                print("equation:", equation)
                if equation not in new_equation_arr:
                    print("insert")
                    new_parent_fit_arr.append(0)
                    new_gen_op_arr.append("creation")
                    new_chrom_arr.append(chromosome)
                    new_equation_arr.append(equation)
                    i += 1
                    done = True

            elif i < (num_reprod + num_create + num_cross):
                # tourn select
                choose_1 = tourn(fit_arr, cross_k)
                chromosome_1 = copy.deepcopy(chrom_arr[choose_1])
                print("best_1:", chrom_arr[choose_1], fit_arr[choose_1])
                choose_2 = tourn(fit_arr, cross_k)
                chromosome_2 = copy.deepcopy(chrom_arr[choose_2])
                print("best_2:", chrom_arr[choose_2], fit_arr[choose_2])

                prev_equation_1 = printEssential(rs, chromosome_1)
                print("prev_equation_1:", prev_equation_1)
                prev_equation_2 = printEssential(rs, chromosome_2)
                print("prev_equation_2:", prev_equation_2)

                # crossover
                chromosome_1, chromosome_2 = crossover(chromosome_1, chromosome_2)
                print("cross_1:", chromosome_1)
                print("cross_2:", chromosome_2)

                equation_1 = printEssential(rs, chromosome_1)
                print("equation_1:", equation_1)
                equation_2 = printEssential(rs, chromosome_2)
                print("equation_2:", equation_2)

                if equation_1 == prev_equation_1 or equation_2 == prev_equation_2:
                    print("no effective change")
                    continue #no change effectively happened so redo


                if equation_1 not in new_equation_arr and equation_2 not in new_equation_arr:
                    print("insert")
                    new_parent_fit_arr.append(fit_arr[choose_1])
                    new_gen_op_arr.append("crossover")
                    new_chrom_arr.append(chromosome_1)
                    new_equation_arr.append(equation_1)

                    print("insert")
                    new_parent_fit_arr.append(fit_arr[choose_2])
                    new_gen_op_arr.append("crossover")
                    new_chrom_arr.append(chromosome_2)
                    new_equation_arr.append(equation_2)
                    i += 2
                    done = True

            elif i < (num_reprod + num_create + num_cross + num_mutate):
                # tourn select
                choose = tourn(fit_arr, mutate_k)
                chromosome = copy.deepcopy(chrom_arr[choose])
                print("best:", chrom_arr[choose], fit_arr[choose])

                prev_equation = printEssential(rs, chromosome)
                print("prev_equation:", prev_equation)

                # mutate
                chromosome = mutate(chromosome)
                print("mut:", chromosome)

                equation = printEssential(rs, chromosome)
                print("equation:", equation)

                if equation == prev_equation:
                    print("no effective change")
                    continue #no change effectively happened so redo

                if equation not in new_equation_arr:
                    print("insert")
                    new_parent_fit_arr.append(fit_arr[choose])
                    new_gen_op_arr.append("mutation")
                    new_chrom_arr.append(chromosome)
                    new_equation_arr.append(equation)
                    i += 1
                    done = True
    
    new_parent_fit_arr, new_gen_op_arr, new_chrom_arr = zip(*sorted(zip(new_parent_fit_arr, new_gen_op_arr, new_chrom_arr), key=lambda k: random.random()))

    if gen != num_gens:
        print(gen, "- refill population with new chromosomes")
        for i in range(num_runners):
            runner_filename = control_path + "runner_" + str(i) + "_evalpop.json"
            runner_file = open(runner_filename)
            runner = json.load(runner_file)
            runner_file.close()

            runner['which_to_eval'] = num_per_runner
            runner['chromosomes'] = [new_chrom_arr[j] for j in range(i*num_per_runner, (i+1)*num_per_runner)]
            runner['gen_ops'] = [new_gen_op_arr[j] for j in range(i*num_per_runner, (i+1)*num_per_runner)]
            runner['parent_fitnesses'] = [new_parent_fit_arr[j] for j in range(i*num_per_runner, (i+1)*num_per_runner)]
            runner['fitnesses'] = [None for x in range(i*num_per_runner, (i+1)*num_per_runner)]
            runner['gen'] = runner['gen']+1
            runner['finished'] = False

            runner_file = open(runner_filename, 'w')
            json.dump(runner, runner_file)
            runner_file.close()

    end_time = time.time()
    print("time in seconds:", (end_time - start_time))



print()
print("FINAL EVALUATION OF BEST")
print()

print(num_gens+1, "- refill population with best ever chromosomes from history")
for i in range(num_runners):
    runner_filename = control_path + "runner_" + str(i) + "_evalpop.json"
    runner_file = open(runner_filename)
    runner = json.load(runner_file)
    runner_file.close()

    runner['which_to_eval'] = num_per_runner
    runner['chromosomes'] = [history_arr[j][3] for j in range(i*num_per_runner, (i+1)*num_per_runner)]
    runner['gen_ops'] = ["history" for j in range(i*num_per_runner, (i+1)*num_per_runner)]
    runner['parent_fitnesses'] = [history_arr[j][1] for j in range(i*num_per_runner, (i+1)*num_per_runner)]
    runner['fitnesses'] = [None for x in range(i*num_per_runner, (i+1)*num_per_runner)]
    runner['gen'] = runner['gen']
    runner['finished'] = False

    runner_file = open(runner_filename, 'w')
    json.dump(runner, runner_file)
    runner_file.close()

end_time = time.time()
print("time in seconds:", (end_time - start_time))


print(num_gens+1, "- tell runners to evaluate population")
changeState('evaluate')


print(num_gens+1, "- wait for eval pops to be finished")
finished = False

count = 0
while not finished:
    time.sleep(random.random() * 3) #seconds

    count += 1
    print(num_gens+1, "- waiting... ", count)

    num_finished = 0
    for i in range(num_runners):
        filename = control_path + "runner_" + str(i) + "_evalpop.json"
        file = open(filename)
        runner = json.load(file)
        file.close()

        if(runner['finished']):
            num_finished += 1
    
    print("num_finished", num_finished)

    if(num_finished == num_runners):
        finished = True

print(num_gens+1, "- finished!")

end_time = time.time()
print("time in seconds:", (end_time - start_time))




print(num_gens+1, "- gather fitnesses")
chrom_arr = []
fit_arr = []
parent_fit_arr = []
gen_op_arr = []

for i in range(num_runners):
    runner_filename = control_path + "runner_" + str(i) + "_evalpop.json"
    runner_file = open(runner_filename)
    runner = json.load(runner_file)
    runner_file.close()

    chrom_arr.extend(runner['chromosomes'])
    fit_arr.extend(runner['fitnesses'])
    parent_fit_arr.extend(runner['parent_fitnesses'])
    gen_op_arr.extend(runner['gen_ops'])

print(chrom_arr)
print(fit_arr)
print(parent_fit_arr)
print(gen_op_arr)


fit_arr, chrom_arr, parent_fit_arr, gen_op_arr = zip(*sorted(zip(fit_arr, chrom_arr, parent_fit_arr, gen_op_arr)))

print(chrom_arr)
print(fit_arr)
print(parent_fit_arr)
print(gen_op_arr)

equation_arr = [printEssential(rs, chromosome) for chromosome in chrom_arr]
print(equation_arr)

for i in range(len(equation_arr)):
    eq = str(equation_arr[i])
    if eq not in member_hash_store:
        new_obj = {"avg": fit_arr[i], "num": 1, "chrom": chrom_arr[i]}
        member_hash_store[eq] = new_obj
    else:
        old_obj = member_hash_store[eq]
        old_avg = old_obj["avg"]
        old_num = old_obj["num"]

        new_num = old_num + 1
        new_avg = ((old_avg * old_num) + fit_arr[i]) / new_num
        new_obj = {"avg": new_avg, "num": new_num, "chrom": chrom_arr[i]}

        member_hash_store[eq] = new_obj

print()
history_arr = []
for key in member_hash_store:
    history_arr.append([key, member_hash_store[key]["avg"], member_hash_store[key]["num"], member_hash_store[key]["chrom"]])
history_arr = sorted(history_arr, key=lambda x: x[1])
for i in range(len(history_arr)):
    ele = history_arr[i]
    print(i, ":", ele[0])
    print(ele[1], ele[2], ele[3])
print()


print("tell runners that they are done")
changeState('finished')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
run_obj = {}

run_obj["params"] = {}

run_obj["params"]["num_runners"] = num_runners
run_obj["params"]["pop_size"] = pop_size
run_obj["params"]["num_gens"] = num_gens
run_obj["params"]["num_per_runner"] = num_per_runner
run_obj["params"]["num_init_per_runner"] = num_init_per_runner


run_obj["params"]["num_create"] = num_create
run_obj["params"]["num_cross"] = num_cross
run_obj["params"]["cross_k"] = cross_k
run_obj["params"]["num_mutate"] = num_mutate
run_obj["params"]["mutate_k"] = mutate_k
run_obj["params"]["num_reprod"] = num_reprod


run_obj["best"] = {}
print("best_chromosome", best_chromosome)
run_obj["best"]["best_chromosome"] = best_chromosome

print("best_fit", best_fit)
run_obj["best"]["best_fit"] = float(best_fit)

print("best_eq", printEssential(rs, best_chromosome))
run_obj["best"]["best_eq"] = printEssential(rs, best_chromosome)


history_arr = []
run_obj["history_arr"] = []
for key in member_hash_store:
    history_arr.append([key, member_hash_store[key]["avg"], member_hash_store[key]["num"], member_hash_store[key]["chrom"]])
history_arr = sorted(history_arr, key=lambda x: x[1])
for i in range(len(history_arr)):
    ele = history_arr[i]
    print(i, ":", ele[0])
    print(ele[1], ele[2], ele[3])
    run_obj["history_arr"].append([i, ele[0], float(ele[1]), ele[2], ele[3]])


end_time = time.time()
print("time in seconds:", (end_time - start_time))
run_obj["time"] = (end_time - start_time)

print("random seed:", seed)
run_obj["seed"] = seed
run_obj["dataset_id"] = dataset_id


run_obj["fit_arrays"] = {}

print("gens", gens)
run_obj["fit_arrays"]["gens"] = gens
print("total_mean_arr", total_mean_arr)
run_obj["fit_arrays"]["total_mean_arr"] = total_mean_arr
print("total_min_arr", total_min_arr)
run_obj["fit_arrays"]["total_min_arr"] = total_min_arr
print("total_1_4_quart_arr", total_1_4_quart_arr)
run_obj["fit_arrays"]["total_1_4_quart_arr"] = total_1_4_quart_arr
print("total_2_4_quart_arr", total_2_4_quart_arr)
run_obj["fit_arrays"]["total_2_4_quart_arr"] = total_2_4_quart_arr
print("total_3_4_quart_arr", total_3_4_quart_arr)
run_obj["fit_arrays"]["total_3_4_quart_arr"] = total_3_4_quart_arr
print("total_std_arr", total_std_arr)
run_obj["fit_arrays"]["total_std_arr"] = total_std_arr
print("total_diff_arr", total_diff_arr)
run_obj["fit_arrays"]["total_diff_arr"] = total_diff_arr


run_obj["genop_arrays"] = {}

print("reprod_to_nan_arr", reprod_to_nan_arr)
run_obj["genop_arrays"]["reprod_to_nan_arr"] = reprod_to_nan_arr
print("create_to_nan_arr", create_to_nan_arr)
run_obj["genop_arrays"]["create_to_nan_arr"] = create_to_nan_arr
print("cross_to_nan_arr", cross_to_nan_arr)
run_obj["genop_arrays"]["cross_to_nan_arr"] = cross_to_nan_arr
print("mutate_to_nan_arr", mutate_to_nan_arr)
run_obj["genop_arrays"]["mutate_to_nan_arr"] = mutate_to_nan_arr

print("reprod_improve_arr", reprod_improve_arr)
run_obj["genop_arrays"]["reprod_improve_arr"] = reprod_improve_arr
print("create_improve_arr", create_improve_arr)
run_obj["genop_arrays"]["create_improve_arr"] = create_improve_arr
print("cross_improve_arr", cross_improve_arr)
run_obj["genop_arrays"]["cross_improve_arr"] = cross_improve_arr
print("mutate_improve_arr", mutate_improve_arr)
run_obj["genop_arrays"]["mutate_improve_arr"] = mutate_improve_arr

print("reprod_decrease_arr", reprod_decrease_arr)
run_obj["genop_arrays"]["reprod_decrease_arr"] = reprod_decrease_arr
print("create_decrease_arr", create_decrease_arr)
run_obj["genop_arrays"]["create_decrease_arr"] = create_decrease_arr
print("cross_decrease_arr", cross_decrease_arr)
run_obj["genop_arrays"]["cross_decrease_arr"] = cross_decrease_arr
print("mutate_decrease_arr", mutate_decrease_arr)
run_obj["genop_arrays"]["mutate_decrease_arr"] = mutate_decrease_arr


run_filename = "d" + str(dataset_id) + "_" + str(seed) + ".json"

run_file = open(run_path + run_filename, 'w')
json.dump(run_obj, run_file)
run_file.close()

best_chromosome ['0000000', '0001100', '0001101', '0001100', '0010001', '0001001', '1000001', '0100100', '0000001', '0011111', '0000001', '0011000', '0000101', '1011111', '0000001', '1010001', '0000001', '0001101', '0000001', '1101000', '0001001', '1100010', '0000001', '0000111', '0000001', '0010111', '0010001', '0010010', '0000001', '1111001', '0110111']
best_fit 0.007785758934915066
best_eq (V) {'a': 14, 'b': 15, 'c': 14, 'i': 10, 'o': 1} $ self.gu ( 'ab' , 0.5 , 'mse' , ['i', 'c', 'ia', 'ib', 'o'] ) ~ self.gu ( 'ac' , 0.1 , 'mse' , ['i', 'a', 'c', 'ib', 'o'] ) ~ self.gu ( 'ao' , 0.1 , 'mse' , ['i', 'b', 'ic', 'ia', 'o'] ) ~ self.gu ( 'ba' , 0.1 , 'mse' , ['i', 'c', 'b', 'a', 'o'] ) ~ self.gu ( 'bc' , 0.9 , 'mse' , ['i', 'a', 'ib', 'ic', 'o'] ) ~ self.gu ( 'bo' , 0.6 , 'mse' , ['i', 'iabc', 'o'] ) ~ self.gu ( 'ca' , 0.1 , 'mse' , ['i', 'c', 'ia', 'b', 'o'] ) ~ self.gu ( 'cb' , 0.1 , 'mse' , ['i', 'a', 'c', 'ib', 'o'] ) ~ self.gu ( 'co' , 0.1 , 'mse' , ['i', 'b', 'ic', 'o'] ) ~ self.g